In [1]:
!export DISABLE_TELEMETRY=YES

In [6]:
import ollama
import pandas as pd
import json

from tqdm import tqdm
#from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

In [3]:
df = pd.read_csv('top10k.csv')

In [4]:
df

,Unnamed: 0,title,selftext,ups
0,74233,TIFU by having my reddit history revealed by J...,This happened to me yesterday and I'm still ba...,85249
1,76661,TIFU by cumming into a coconut,EDIT: I got an AMA thread now. Help me: \n\nht...,81547
2,42440,TIFU by stuffing my face with edibles before d...,"Recently, I traveled to Denver, Colorado with ...",80916
3,35184,TIFU by getting Reddit banned in Russia,"Today Reddit was blocked in Russia, and I am t...",64262
4,73236,TIFU by not telling my wife our son was coming...,This happened Sunday night.\n\n\n\nMy oldest s...,49877
...,...,...,...,...
9995,42961,TIFUpdate by making a dumb joke to myself whic...,[Some of you may remember my stupid joke](http...,130
9996,71249,TIFU by pressing the wrong button,"So by some stroke of luck, I was invited to at...",130
9997,73043,TIFU my first edible experience,"Mandatory ""This actually happened about a year...",130
9998,36945,TIFU by holding the door open for an old man a...,I'm still stunned and have no idea what I did ...,130


In [15]:
def check_vibes(storytext):
    try:
        tries = 0
        while tries < 3:
            approvaljson = ollama.generate(model='llama3:instruct', 
                                     format='json',
                                 prompt='''You are the world's best editor you are choosing stories to adapt to short videos, read the following story and decide if it is pg-13 and would appeal to a wide audience. Avoid approving stories that are overly sexual in nature. give your decision in the following json format: {"reasoning":"provide a reasoning for your descision here", "approval":true/false} .
                                 here's the story: ''' + storytext)
            approvaldict = json.loads(approvaljson['response'])
            if 'approval' in approvaldict.keys():
                # if 'reasoning' in approvaldict.keys():
                #     print(approvaldict['reasoning'])
                return approvaldict['approval']
            else:
                tries = tries + 1
        return False
    except:
        return False
    

In [16]:
df['approval'] = df.progress_apply(lambda row: check_vibes(row.selftext), axis = 1)

100%|█████████████████████████████████████████████████████████████████████| 10000/10000 [3:30:28<00:00,  1.26s/it]


In [18]:
df['approval'].describe()


count     10000
unique        2
top        True
freq       7333
Name: approval, dtype: object

In [17]:
df.to_csv('top10k_approval.csv')

In [20]:
df = df[df.approval == True]

In [23]:
pd.set_option('display.max_colwidth', None)

In [24]:
df.selftext.head(10)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [25]:
df.to_csv('approved_stories.csv')